In [ ]:
#input must be options chain day1 - end

In [ ]:
import wandb
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import joblib
import random

In [ ]:
CSV_PATH     = './data/OptionsEOD.csv/'
PARQUET_PATH = './data/OptionsEOD.parquet'
PARQUET_STG_PATH = './data/OptionsEOD_STG.parquet'
SCALER_COL  = ['DTE','INTRINSIC_VALUE','TOTAL_C_VOLUME', 'TOTAL_P_VOLUME',	'C_BID',	'C_ASK', 'C_VOLUME',  'P_BID',	'P_ASK',	'P_VOLUME' ]
SCALER_PATH = './data/scaler.gz'
 
UNIQUE_KEYS = ['QUOTE_DATE','SYMBOL','EXPIRE_DATE']

In [ ]:
#example
EOD_CSV = pd.read_csv(CSV_PATH+"qqq/qqq_eod_201201.txt", engine='pyarrow')
EOD_CSV.head()

In [ ]:
EOD_CSV.columns

In [ ]:
#Part I
#TransformData : 
#-each partition from EXPIRE_DATE 
#-csv too parquet
#-col. rename 
def TransformDataI():
    scaler = MinMaxScaler()
    schema = None
    pqwriter = None
    for d in os.listdir(CSV_PATH):
        for f in os.listdir(CSV_PATH+f"{d}/"):
            if f.endswith(".txt"):
                ## load
                print( f"[LOAD] : {CSV_PATH}{d}/{f}        ",end='\r')
                EOD_CSV = pd.read_csv(CSV_PATH+f"{d}/"+f, engine='pyarrow')
                    
                ## rename col.
                for c in EOD_CSV.columns:
                    EOD_CSV = EOD_CSV.rename( columns={ c:c.strip().replace(']','').replace('[','') } )
                
                ## add symbol 
                EOD_CSV['SYMBOL'] = d.upper()
                ## add INTRINSIC_VALUE
                EOD_CSV['INTRINSIC_VALUE'] = EOD_CSV['UNDERLYING_LAST'] - EOD_CSV['STRIKE']
                
                ## fillnafillna
                EOD_CSV['P_VOLUME'] = EOD_CSV['P_VOLUME'].fillna(0)
                EOD_CSV['C_VOLUME'] = EOD_CSV['C_VOLUME'].fillna(0)
                
                # date columns convert to datetime
                for c in ["QUOTE_READTIME","QUOTE_DATE","EXPIRE_DATE"]:
                    EOD_CSV[c] = pd.to_datetime(EOD_CSV[c])
                
                #clean float data
                for c in ['INTRINSIC_VALUE','C_DELTA','C_GAMMA','C_VEGA','C_THETA','C_RHO','C_IV','C_VOLUME','C_LAST','C_BID','C_ASK','STRIKE','P_BID','P_ASK','P_LAST','P_DELTA','P_GAMMA','P_VEGA','P_THETA','P_RHO','P_IV','P_VOLUME','STRIKE_DISTANCE','STRIKE_DISTANCE_PCT']:
                    if EOD_CSV[c].dtype not in ( 'float32','float64'):
                        EOD_CSV[c] = EOD_CSV[c].apply(lambda x: x.strip())
                        EOD_CSV[c] = EOD_CSV[c].replace('', np.nan).fillna(np.nan)
                        EOD_CSV[c] = EOD_CSV[c].astype('float64')
                    if EOD_CSV[c].dtype == 'float32':
                        EOD_CSV[c] = EOD_CSV[c].astype('float64')
                        
                # REMAIN_DAYS(int) =>  use DTE col.
                #partition with QUOTE_DATE
                EOD_CSV['PartitionDate'] = EOD_CSV['QUOTE_DATE'].dt.strftime('%Y-%m')
                EOD_CSV.sort_values(['QUOTE_DATE','EXPIRE_DATE','SYMBOL','STRIKE'],ascending =False ) 

                #scaler(Normalization_
                #scaler.partial_fit(EOD_CSV[SCALER_COL])

                # save
                if os.path.exists(PARQUET_PATH):
                  EOD_CSV.to_parquet(PARQUET_PATH, engine='fastparquet', append=True, partition_cols=['PartitionDate'], index=False )
                else:
                  EOD_CSV.to_parquet(PARQUET_PATH, engine='fastparquet' , partition_cols=['PartitionDate'], index=False  )
                    
    # joblib.dump(scaler, SCALER_PATH )
    # if pqwriter:
    #     pqwriter.close()
    # print( f"[DONE]                                                       ",end='\r')



In [ ]:
##-RunCleanData
#TransformDataI()

In [ ]:
#Part II 
#TransformData : 
# - read each partitions 
# - Normalization if not have scaler.gz file
def strikeZero(df,v):
    # First, filter based on QUOTE_DATE, SYMBOL, and EXPIRE_DATE
    filtered_df = df[(df['QUOTE_DATE'] == v['QUOTE_DATE']) &
                     (df['SYMBOL'] == v['SYMBOL']) &
                     (df['EXPIRE_DATE'] == v['EXPIRE_DATE'])]
    
    # Then, find the max and min INTRINSIC_VALUE within the filtered subset
    max_value = filtered_df['STRIKE'].max()
    min_value = filtered_df['STRIKE'].min()
    # print(max_intrinsic_value)
    # print(min_intrinsic_value)
    df.loc[ (df['QUOTE_DATE'] == v['QUOTE_DATE']) 
            & (df['SYMBOL'] == v['SYMBOL']) 
            & (df['EXPIRE_DATE'] == v['EXPIRE_DATE'])
            & (  (df['STRIKE'] == max_value)
               | (df['STRIKE'] == min_value)
              ) 
        , ['DTE', 'INTRINSIC_VALUE', 'C_BID',	'C_ASK', 'C_VOLUME',  'P_BID',	'P_ASK',	'P_VOLUME']       
    ] = 0
    
def TransformDataII():
    
    keys = None#df[unique_keys].sort_values(by=unique_keys).drop_duplicates()
    max_option_len = 20
    scaler = MinMaxScaler()
    PartitionDate = [ d[-7:] for d in  os.listdir(PARQUET_PATH) if 'PartitionDate' in d]
    options_qoute = {}
    for i,partdate in enumerate(PartitionDate) :  
        df = pd.read_parquet(PARQUET_PATH,engine='pyarrow'
                                     , filters=[('PartitionDate', '=', partdate)]
                                    )
        ####################################################
        keys = df[UNIQUE_KEYS].sort_values(by=UNIQUE_KEYS).drop_duplicates()
        #loop each keys
        for j,v in keys.iterrows():
            
            df_filter=df[ (df['QUOTE_DATE'] == v['QUOTE_DATE']) & (df['SYMBOL'] == v['SYMBOL']) & (df['EXPIRE_DATE'] == v['EXPIRE_DATE']) ]
            qoute = "".join(v[ ['SYMBOL','EXPIRE_DATE'] ].apply(str).values)
            #add qoute
            if qoute not in [*options_qoute.keys()]:
                options_qoute[qoute] = {}
                options_qoute[qoute]['start_price'] = df_filter['UNDERLYING_LAST'].values[0]
                options_qoute[qoute]['strike'] = df_filter[ df_filter['INTRINSIC_VALUE'].abs().isin(df_filter['INTRINSIC_VALUE'].abs().sort_values()[:max_option_len]) ]['STRIKE'].values
                options_qoute[qoute]['exp'] = df_filter['EXPIRE_DATE'].values[0]
                #check diff UNDERLYING_LAST
                if df_filter['UNDERLYING_LAST'].values[0] != round(np.average(df_filter['UNDERLYING_LAST']),4):
                    print('[ERROR] : set UNDERLYING_LAST ',qoute )
            #rm index max : max_option_len
            rm_strike_index = df_filter[ ~df_filter['STRIKE'].isin(options_qoute[qoute]['strike']) ].index
            df = df.drop(rm_strike_index)
            df_filter = df_filter.drop(rm_strike_index)

            # Generate zero strike 
            # Generate a random float between 0.01 and 1
            random_number = random.uniform(0, 1)
            [ strikeZero(df,v) for r in [0.05,0.1,0.15,0.2,0.25,0.3] if random_number < r]
                
            #TOTAL_P_VOLUME
            df.loc[ (df['QUOTE_DATE'] == v['QUOTE_DATE']) 
            & (df['SYMBOL'] == v['SYMBOL']) 
            & (df['EXPIRE_DATE'] == v['EXPIRE_DATE']) 
            , 'TOTAL_P_VOLUME'] = df_filter['P_VOLUME'].sum()
        
            #TOTAL_P_VOLUME
            df.loc[ (df['QUOTE_DATE'] == v['QUOTE_DATE']) 
            & (df['SYMBOL'] == v['SYMBOL']) 
            & (df['EXPIRE_DATE'] == v['EXPIRE_DATE']) 
            , 'TOTAL_C_VOLUME'] = df_filter['C_VOLUME'].sum()
    
        #clear expire options_qoute
        for qi in list(options_qoute.keys()):
            if options_qoute[qi]['exp'] < df['QUOTE_DATE'].values[0]:
                options_qoute.pop(qi)
    
        #filled na
        df['P_VOLUME'] = df['P_VOLUME'].fillna(0)
        df['C_VOLUME'] = df['C_VOLUME'].fillna(0)
        ###################################################

        # scaler.partial_fit
        scaler.partial_fit(df[SCALER_COL])
        print(f"[Processing] {round(((i+1)/len(PartitionDate))*100,2)}%   ",end='\r')
        
        # save
        if os.path.exists(PARQUET_STG_PATH):
          df.to_parquet(PARQUET_STG_PATH, engine='fastparquet', append=True, partition_cols=['PartitionDate'], index=False )
        else:
          df.to_parquet(PARQUET_STG_PATH, engine='fastparquet' , partition_cols=['PartitionDate'], index=False  )
            
    joblib.dump(scaler, SCALER_PATH )

In [ ]:
TransformDataII()

In [ ]:
#Example data
PartitionDate = [ d[-7:] for d in  os.listdir(PARQUET_STG_PATH) if 'PartitionDate' in d]
options_qoute = {}
for i,partdate in enumerate(PartitionDate[5:6]) :  
    df = pd.read_parquet(PARQUET_PATH,engine='pyarrow'
                                 , filters=[('PartitionDate', '=', partdate)]
                                )
    break

In [ ]:
keys = df[UNIQUE_KEYS].sort_values(by=UNIQUE_KEYS).drop_duplicates()
#loop each keys
for j,v in keys.iterrows():

    df_filter=df[ (df['QUOTE_DATE'] == v['QUOTE_DATE']) 
        & (df['SYMBOL'] == v['SYMBOL']) 
        & (df['EXPIRE_DATE'] == v['EXPIRE_DATE']) 
    ]

    df_filter=df_filter[
        (df_filter['INTRINSIC_VALUE'].max() == df_filter['INTRINSIC_VALUE'])
    |   (df_filter['INTRINSIC_VALUE'].min() == df_filter['INTRINSIC_VALUE'])
    ]
    break

In [ ]:
3============== sampling show 3===========================

In [ ]:
PartitionDate = [ d[-7:] for d in  os.listdir(PARQUET_STG_PATH) if 'PartitionDate' in d]

In [ ]:
import random
from IPython.display import clear_output,display, HTML

for i in range(10):
    ran_partiion=random.choice(PartitionDate)

    df = pd.read_parquet(PARQUET_STG_PATH,engine='pyarrow'
                                 , filters=[('PartitionDate', '=', ran_partiion)]
                                )
    key = random.choice( [*df[UNIQUE_KEYS].sort_values(by=UNIQUE_KEYS).drop_duplicates().values] )

    df_filter=df[ (df['QUOTE_DATE'] == key[0]) 
        & (df['SYMBOL'] == key[1]) 
        & (df['EXPIRE_DATE'] == key[2]) 
    ]
    display(HTML(df_filter[['STRIKE']+SCALER_COL].to_html()))
    input('Next ...')
    clear_output()
